# NumPy World

In [195]:
import numpy as np

In [196]:
N = 10
D = 5
NB_CLASSES = 4

In [205]:
Th = np.random.random((N, D))*1
x = np.random.random(D)
logits = Th.dot(x)

In [206]:
SCALE = 2.5
mini = logits.mean() - SCALE * (logits.mean() - logits.min())
maxi = logits.mean() + SCALE * (logits.max() - logits.mean())
thresholds = np.linspace(mini, maxi, NB_CLASSES - 1)
thresholds

array([-0.34937362,  0.91097076,  2.17131515])

In [207]:
labels = np.round(np.random.random(N)*3)
labels

array([ 2.,  1.,  1.,  2.,  0.,  2.,  2.,  3.,  2.,  3.])

In [208]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [209]:
cdf = sigmoid(Th.dot(x)[:, None] - thresholds)
cdf

array([[ 0.81562349,  0.55641588,  0.26236434],
       [ 0.82588572,  0.57356227,  0.27608919],
       [ 0.86533386,  0.64564997,  0.34065603],
       [ 0.78178965,  0.50394538,  0.22364281],
       [ 0.81308505,  0.55226776,  0.2591278 ],
       [ 0.82502042,  0.5720927 ,  0.27489048],
       [ 0.87201811,  0.65894066,  0.35393945],
       [ 0.86492973,  0.64485715,  0.3398785 ],
       [ 0.78413529,  0.50739579,  0.2260486 ],
       [ 0.71313231,  0.41345553,  0.16658266]])

In [210]:
operator = np.fromfunction(lambda i, j: (j == i + 1) - 1 * (j == i), (NB_CLASSES - 1, NB_CLASSES), dtype=float)

In [211]:
b = np.fromfunction(lambda i, j: 1. * (j == 0), (1, NB_CLASSES), dtype=float)

In [212]:
pdf = cdf.dot(operator) + b

In [213]:
pdf

array([[ 0.18437651,  0.25920761,  0.29405153,  0.26236434],
       [ 0.17411428,  0.25232345,  0.29747308,  0.27608919],
       [ 0.13466614,  0.21968389,  0.30499394,  0.34065603],
       [ 0.21821035,  0.27784427,  0.28030257,  0.22364281],
       [ 0.18691495,  0.26081729,  0.29313996,  0.2591278 ],
       [ 0.17497958,  0.25292772,  0.29720222,  0.27489048],
       [ 0.12798189,  0.21307745,  0.30500121,  0.35393945],
       [ 0.13507027,  0.22007258,  0.30497864,  0.3398785 ],
       [ 0.21586471,  0.2767395 ,  0.28134719,  0.2260486 ],
       [ 0.28686769,  0.29967678,  0.24687288,  0.16658266]])

In [214]:
pred_labels = np.argmax(pdf, axis=1)
pred_labels

array([2, 2, 3, 2, 2, 2, 3, 3, 2, 1])

In [216]:
from sklearn.metrics import log_loss
log_loss(labels, pdf)

1.360597513845669

In [151]:
#sess = tf.InteractiveSession()
#sess.run(tf.argmax(pdf, axis=1))

In [152]:
def logloss(x):
    return np.log(1 + np.exp(-x))

<tf.Tensor 'logistic_loss:0' shape=(10,) dtype=float64>

In [135]:
sgn = np.fromfunction(lambda i, j: 2 * (1 + j < labels[i]) - 1, (N, L - 1), dtype=int)

In [136]:
delta = np.outer(Th.dot(x), np.ones(3)) - thresholds

In [137]:
logloss(sgn * delta).sum(axis=1)

array([ 5.65487554,  7.17739489,  6.20764354,  5.56366886,  3.69574892,
        3.71039728,  2.38339264,  6.32255626,  7.24467443,  3.3684874 ,
        5.26491588,  7.8594586 ,  5.86722618,  2.31494799,  7.42424508,
        4.26048122,  6.98919102,  2.80930126,  4.96971201,  6.36932063])

# TensorFlow World

In [138]:
import tensorflow as tf

aa = tf.constant([1., 5., 3.])
a = tf.reshape(aa, (3, 1))
b = tf.ones([1, 3])

In [139]:
#sess = tf.InteractiveSession()
with tf.Session() as sess:
    print(sess.run(tf.matmul(a, b)))

[[ 1.  1.  1.]
 [ 5.  5.  5.]
 [ 3.  3.  3.]]


In [141]:
some_logits[:, None] - thresholds

array([[ 2.60112042,  2.72313403,  2.7336119 ],
       [ 2.21160787,  2.33362148,  2.34409935],
       [ 1.8491477 ,  1.97116131,  1.98163918],
       [ 1.59921351,  1.72122712,  1.73170499],
       [ 0.80161644,  0.92363005,  0.93410792],
       [ 1.42925511,  1.55126872,  1.56174659],
       [ 1.84278532,  1.96479893,  1.9752768 ],
       [ 2.96445091,  3.08646452,  3.09694239],
       [ 3.45197331,  3.57398692,  3.58446479],
       [ 1.1853049 ,  1.30731851,  1.31779638],
       [ 2.38295263,  2.50496624,  2.51544411],
       [ 2.45924797,  2.58126158,  2.59173945],
       [ 1.71812721,  1.84014082,  1.85061869],
       [ 1.72850927,  1.85052288,  1.86100075],
       [ 2.30181929,  2.4238329 ,  2.43431077],
       [ 1.78828355,  1.91029716,  1.92077503],
       [ 3.31815945,  3.44017305,  3.45065092],
       [ 0.35326872,  0.47528233,  0.4857602 ],
       [ 2.21388236,  2.33589597,  2.34637384],
       [ 1.91060683,  2.03262044,  2.04309831]])

In [147]:
2 * (range(4) - labels[:, None]) + 1

array([[-3., -1.,  1.,  3.],
       [-1.,  1.,  3.,  5.],
       [-1.,  1.,  3.,  5.],
       [-1.,  1.,  3.,  5.],
       [-1.,  1.,  3.,  5.],
       [-3., -1.,  1.,  3.],
       [-5., -3., -1.,  1.],
       [-3., -1.,  1.,  3.],
       [-3., -1.,  1.,  3.],
       [-3., -1.,  1.,  3.],
       [-3., -1.,  1.,  3.],
       [-1.,  1.,  3.,  5.],
       [ 1.,  3.,  5.,  7.],
       [-5., -3., -1.,  1.],
       [-1.,  1.,  3.,  5.],
       [-3., -1.,  1.,  3.],
       [-3., -1.,  1.,  3.],
       [-1.,  1.,  3.,  5.],
       [-3., -1.,  1.,  3.],
       [-1.,  1.,  3.,  5.]])

In [221]:
logi = tf.placeholder(tf.float32, shape=[None])
thr = tf.placeholder(tf.float32, shape=[None])
lab = tf.placeholder(tf.int32, shape=[None])

In [321]:
logi = tf.constant(logits)
thr = tf.constant(thresholds)
lab_f = tf.constant(labels, dtype=tf.float32)
lab_i = tf.constant(labels, dtype=tf.int32)

In [314]:
tf_pdf = tf.constant(pdf, dtype=tf.float32)
tf_cdf = tf.constant(cdf, dtype=tf.float32)

In [319]:
pdf_logits = tf.log(tf_pdf / (1 - tf_pdf))
cdf_logits = tf.log(tf_cdf / (1 - tf_cdf))

In [322]:
labels_pdf = tf.one_hot(lab_i, depth=NB_CLASSES)

In [340]:
labels_cdf = tf.nn.relu(tf.sign(lab_f[:, None] + 0.5 - tf.range(1, NB_CLASSES, dtype=tf.float32)))

In [341]:
labels

array([ 2.,  1.,  1.,  2.,  0.,  2.,  2.,  3.,  2.,  3.])

In [330]:
pdf

array([[ 0.18437651,  0.25920761,  0.29405153,  0.26236434],
       [ 0.17411428,  0.25232345,  0.29747308,  0.27608919],
       [ 0.13466614,  0.21968389,  0.30499394,  0.34065603],
       [ 0.21821035,  0.27784427,  0.28030257,  0.22364281],
       [ 0.18691495,  0.26081729,  0.29313996,  0.2591278 ],
       [ 0.17497958,  0.25292772,  0.29720222,  0.27489048],
       [ 0.12798189,  0.21307745,  0.30500121,  0.35393945],
       [ 0.13507027,  0.22007258,  0.30497864,  0.3398785 ],
       [ 0.21586471,  0.2767395 ,  0.28134719,  0.2260486 ],
       [ 0.28686769,  0.29967678,  0.24687288,  0.16658266]])

In [339]:
cdf
cdf_logits.eval(), labels_cdf.eval(), lab_i.eval()

(array([[ 1.48697281,  0.2266285 , -1.03371572],
        [ 1.55674446,  0.29640013, -0.9639442 ],
        [ 1.86031663,  0.59997231, -0.66037202],
        [ 1.27612627,  0.01578174, -1.24456251],
        [ 1.47018182,  0.2098375 , -1.05050683],
        [ 1.55073893,  0.29039451, -0.96994996],
        [ 1.91892135,  0.65857702, -0.60176742],
        [ 1.85685313,  0.5965088 , -0.66383576],
        [ 1.28992963,  0.0295854 , -1.23075902],
        [ 0.91064596, -0.34969845, -1.61004293]], dtype=float32),
 array([[ 1.,  1.,  0.],
        [ 1.,  0.,  0.],
        [ 1.,  0.,  0.],
        [ 1.,  1.,  0.],
        [ 0.,  0.,  0.],
        [ 1.,  1.,  0.],
        [ 1.,  1.,  0.],
        [ 1.,  1.,  1.],
        [ 1.,  1.,  0.],
        [ 1.,  1.,  1.]], dtype=float32),
 array([2, 1, 1, 2, 0, 2, 2, 3, 2, 3], dtype=int32))

In [305]:
def tf_cross_entropy(p, labels_oh):
    return -tf.reduce_mean(tf.reduce_sum(tf.multiply(labels_oh, tf.log(p)), axis=1))

In [342]:
with tf.Session() as sess:
    print(sess.run(tf.nn.sigmoid_cross_entropy_with_logits(logits=tf_logits, labels=labels_oh)))
    print(sess.run(tf.nn.sigmoid_cross_entropy_with_logits(logits=cdf_logits, labels=labels_cdf)))

[[ 0.20380244  0.30003488  1.22400022  0.30430529]
 [ 0.19129889  1.37704349  0.35307154  0.3230871 ]
 [ 0.14463989  1.51556563  0.36383468  0.41650996]
 [ 0.24616954  0.3255145   1.27188563  0.2531426 ]
 [ 1.67710161  0.30221015  0.34692261  0.29992712]
 [ 0.19234712  0.29159334  1.21334243  0.32143256]
 [ 0.1369451   0.23962544  1.18743956  0.43686202]
 [ 0.14510702  0.24855442  0.36381269  1.07916713]
 [ 0.2431737   0.32398582  1.26816583  0.25624621]
 [ 0.33808827  0.3562133   0.28352123  1.79226363]]
[[ 0.20380247  0.58623928  0.30430529]
 [ 0.19129889  0.8522889   0.3230871 ]
 [ 0.14463989  1.0374701   0.41650996]
 [ 0.24616954  0.68528748  0.2531426 ]
 [ 1.67710137  0.80355984  0.29992712]
 [ 0.19234712  0.55845422  0.32143256]
 [ 0.1369451   0.41712177  0.43686202]
 [ 0.14510702  0.43872643  1.07916713]
 [ 0.24317373  0.67846388  0.25624621]
 [ 0.33808827  0.88320529  1.79226363]]


In [268]:
with tf.Session() as sess:
    print(sess.run(tf_cross_entropy(tf_pdf, labels_oh)))

1.3606


In [263]:
bla

-1.3605975

In [254]:
bla.mean()

-0.3401494

In [159]:
def tf_logloss(x):
    return tf.log(1 + tf.exp(-x))

In [172]:
def immediate_thresholds(labels, logits, thresholds):
    signs = tf.one_hot(labels, NB_CLASSES - 1) - tf.one_hot(labels - 1, NB_CLASSES - 1)
    delta = logits[:, None] - thresholds
    return tf.reduce_sum(tf_logloss(signs * delta), axis=1)

In [177]:
signs = tf.one_hot(labels, NB_CLASSES - 1) - tf.one_hot(labels - 1, NB_CLASSES - 1)

In [178]:
signs = tf.sign(2 * (tf.range(NB_CLASSES, dtype=tf.float32) - labels[:, None]) + 1)

In [164]:
def all_thresholds(labels, logits, thresholds):
    #batch_size = tf.size(labels).eval()
    #sgn = np.fromfunction(lambda i, j: 2 * (1 + j < labels.eval()[i]) - 1, (batch_size, NB_CLASSES - 1), dtype=int)
    signs = tf.sign(2 * (tf.range(NB_CLASSES, dtype=tf.float32) - labels[:, None]) + 1)
    delta = logits[:, None] - thresholds
    return tf.reduce_sum(tf_logloss(signs * delta), axis=1)

In [166]:
metrics = immediate_thresholds(lab, logits, thr)

In [127]:
np.random.random((5))[None, :].shape

(1, 5)

In [132]:
delta = logits[:, None] - thr

In [179]:
with tf.Session() as sess:
    print(sess.run(signs, feed_dict={lab: labels, logits: some_logits, thr: thresholds}))

[[-1. -1.  1.  1.]
 [-1.  1.  1.  1.]
 [-1.  1.  1.  1.]
 [-1.  1.  1.  1.]
 [-1.  1.  1.  1.]
 [-1. -1.  1.  1.]
 [-1. -1. -1.  1.]
 [-1. -1.  1.  1.]
 [-1. -1.  1.  1.]
 [-1. -1.  1.  1.]
 [-1. -1.  1.  1.]
 [-1.  1.  1.  1.]
 [ 1.  1.  1.  1.]
 [-1. -1. -1.  1.]
 [-1.  1.  1.  1.]
 [-1. -1.  1.  1.]
 [-1. -1.  1.  1.]
 [-1.  1.  1.  1.]
 [-1. -1.  1.  1.]
 [-1.  1.  1.  1.]]


In [176]:
labels

array([ 2.,  1.,  1.,  1.,  1.,  2.,  3.,  2.,  2.,  2.,  2.,  1.,  0.,
        3.,  1.,  2.,  2.,  1.,  2.,  1.])

In [39]:
result = all_thresholds(logits, thr, lab)

ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`

In [46]:
with tf.Session() as sess:
    print(sess.run(all_thresholds(lab, logits, thr), feed_dict={lab: labels, logits: Th.dot(x), thr: thresholds}))

[ 1.6303095   1.20975196  1.60408551  1.75802107  1.24029919  1.88728056
  0.76261534  2.5268018   1.38458055  2.42626421  3.34016564  1.92546685
  0.98619173  4.00346476  2.00474428  1.57902     2.66738516  1.99304748
  3.28847223  1.96645723]
